In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import sys,os,glob,json,pickle
from collections import OrderedDict

import libstempo as T2
import libstempo.toasim as LT
import libstempo.plot as LP

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import utils
from enterprise.signals import gp_signals
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection

from enterprise_extensions import models,model_utils

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

%matplotlib inline

# Names and Directories

In [3]:
current_path = os.getcwd()
splt_path = current_path.split("/")

top_path_idx = splt_path.index('nanograv')
top_dir = "/".join(splt_path[0:top_path_idx+1])

backgrouninjection_dir = top_dir + '/NANOGrav/BackgroundInjections'
pta_sim_dir = top_dir + '/pta_sim/pta_sim'

runname = '/simGWB_only1gwb_2'
#Where the everything should be saved to (chains,etc.)
simdir = current_path + '/SimRuns'
outdir = simdir + runname
if os.path.exists(simdir) == False:
    os.mkdir(simdir)
if os.path.exists(outdir) == False:
    os.mkdir(outdir)

#The pulsars
psrs_wn_only_dir = backgrouninjection_dir + '/FakePTA/'
noise11yr_path = backgrouninjection_dir + '/nano11/noisefiles_new/'
psrlist11yr_path = backgrouninjection_dir + '/nano11/psrlist_Tg3yr.txt'

#### Load Jeff's sim_gw function from pta_sim

In [4]:
sys.path.insert(0,pta_sim_dir)
import sim_gw as SG

#### Get par and tim files

In [5]:
parfiles = sorted(glob.glob(psrs_wn_only_dir+'*.par'))
timfiles = sorted(glob.glob(psrs_wn_only_dir+'*.tim'))

#### Instantiate a "Simulation class"

In [6]:
sim1 = SG.Simulation(parfiles,timfiles)

PSR J2317+1439 loaded.


#### Inject 2 backgrounds

In [7]:
background_amp_1 = 1.3e-15
background_amp_2 = 5.0e-15

background_gamma_1 = 13./3.
background_gamma_2 = 7./3.
background_seed_1 = 1986
background_seed_2 = 1667

#sim1.createGWB(background_amp_1,background_gamma_1,seed=background_seed_1)
sim1.createGWB(background_amp_2,background_gamma_2,seed=background_seed_2)

### Get pulsars as enterprise pulsars

In [8]:
sim1.init_ePulsars()

#### Use Simple 2 GWB model to instantiate enterprise PTA

In [9]:
background_gammas = [background_gamma_1, background_gamma_2]
#pta1 = SG.model_simple_multiple_gwbs(sim1.psrs,gammas=background_gammas)
pta1 = SG.model_simple_multiple_gwbs(sim1.psrs,gammas=[background_gamma_2])

gw_log10_A_1 gw_gamma_1 background_1


#### Save params for plotting

In [10]:
with open(outdir + '/parameters.json', 'w') as fp:
    json.dump(pta1.param_names, fp)

#### Set up sampler and initial samples

In [11]:
#Pick random initial sampling
xs1 = {par.name: par.sample() for par in pta1.params}

# dimension of parameter space
ndim1 = len(xs1)

# initial jump covariance matrix
cov1 = np.diag(np.ones(ndim1) * 0.01**2)

groups1 = model_utils.get_parameter_groups(pta1)
groups1.append([ndim1-2,ndim1-1])

# intialize sampler
sampler = ptmcmc(ndim1, pta1.get_lnlikelihood, pta1.get_lnprior, cov1, groups=groups1, outDir = outdir,resume=False)

# Sample!

In [12]:
# sampler for N steps
N = int(1e5)
x0 = np.hstack(p.sample() for p in pta1.params)
#sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

Finished 1.00 percent in 6.805298 s Acceptance rate = 0.642

/home/andrew/anaconda3/envs/enterprise/lib/python3.6/site-packages/enterprise/signals/parameter.py:66: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 10.00 percent in 66.998420 s Acceptance rate = 0.19763Adding DE jump with weight 50
Finished 99.00 percent in 667.980961 s Acceptance rate = 0.390828
Run Complete
